Microgel size estimation using cellpose

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, measure
from skimage.transform import resize
from cellpose import models
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import Image
from ipyfilechooser import FileChooser
import matplotlib.pyplot as plt

# Select cellpose model
model = models.Cellpose(model_type='cyto3')

# show the graphs and images in the notebook (for VSCode mainly)
%matplotlib inline

# Choosing the folder with the images
def get_folder_chooser():
    fc = FileChooser()
    fc.title = '<b>Select the folder containing your images</b>'
    fc.show_only_dirs = True
    display(fc)
    return fc


# Creating the mask overlay for display
def create_mask_overlay(image, mask):
    plt.figure(figsize=(7, 7))
    plt.imshow(image, cmap='gray')
    plt.imshow(mask, alpha=0.5, cmap='hsv')
    plt.axis('off')
    return plt.gcf()


# Core routine, processing the image through the pipeline
#def process_images(folder_path, progress_bar, output_area, diameter=input_box.value/2):
def process_images(folder_path, progress_bar, output_area):
    
    results = []
    overlay_dir = os.path.join(folder_path, 'mask_overlays')
    os.makedirs(overlay_dir, exist_ok=True)
    
    # Get list of image files
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.tif', '.tiff', '.png'))]
    total_images = len(image_files)
    
    for i, filename in enumerate(image_files):
        img_path = os.path.join(folder_path, filename)
        img = io.imread(img_path)
        # Resizing image
        # Get the original image dimensions
        height, width = img.shape[:2] # this means that the img.shape is returning more than 2 values (x,y,channels) but we only care about the first two

        # Calculate the new dimensions
        new_height = int(height * 0.5)
        new_width = int(width * 0.5)

        # Resize the image using skimage.transform.resize
        img = resize(img, (int(new_height), int(new_width)))
           
        # Run Cellpose with error handling
        try:
            masks, flow, styles = models.CellposeModel(model_type='cyto3').eval(img,
                            diameter=int(diameter/2), channels=[0,0], flow_threshold=0.2)
        except ValueError as e:
            with output_area:
                print(f"Error processing {filename}: {str(e)}")
            continue
            
        progress = (i + 1) / total_images
        progress_bar.value = progress 
        with output_area:
            clear_output(wait=True)
            print(f"Processed image {i+1} of {total_images}: {filename}") 
        

        
        # Create and save mask overlay
        fig = create_mask_overlay(img, masks)
        plt.show(fig)
        overlay_path = os.path.join(overlay_dir, f"{os.path.splitext(filename)[0]}_overlay.png")
        fig.savefig(overlay_path, dpi=300, bbox_inches='tight')
        plt.close(fig)
        
        # Analyze ROIs
        props = measure.regionprops(masks)
        
        for j, prop in enumerate(props):
            results.append({
                'filename': filename,
                'roi_index': j,
                'diameter': prop.equivalent_diameter,
                'area': prop.area,
                'major_axis_length': prop.major_axis_length,
                'minor_axis_length': prop.minor_axis_length,
                'aspect_ratio': prop.major_axis_length / prop.minor_axis_length if prop.minor_axis_length != 0 else np.nan
            })
    
    df=pd.DataFrame(results)
    filtered_df = df[(df['aspect_ratio'] >= 0.8) & (df['aspect_ratio'] <= 1.2)]
    return filtered_df

# Cell 1: Get folder chooser
folder_chooser = get_folder_chooser()

# Cell 2: Process images and export results
def process_and_export(b):
    folder_path = folder_chooser.selected_path
    if not folder_path or not os.path.isdir(folder_path):
        with output_area:
            print("Please select a valid folder.")
        return
    
    with output_area:
        print(f"Processing images in {folder_path}...")   
   
          
    results_df = process_images(folder_path, progress_bar, output_area)  
    
    if results_df.empty:
        print("No results to export. Please check if there were any errors during processing and if you have images on the selected folder.")
        return
    
    output_path = os.path.join(folder_path, 'segmentation_results.xlsx')
    results_df.to_excel(output_path, index=False)
    print(f"Results exported to {output_path}")
    print(f"Mask overlays saved in {os.path.join(folder_path, 'mask_overlays')}")

process_button = widgets.Button(description="Process Images")
output_area=widgets.Output()
progress_bar = widgets.FloatProgress(min=0, max=1, description='Progress:', bar_style='info') 
input_box = widgets.IntText(value=100, description='Est. diam.', disabled=False)
diameter = input_box.value

def on_button_clicked(b):
    diameter = input_box.value
    process_and_export(b)

process_button.on_click(on_button_clicked)
display(process_button)

# display(widgets.VBox([progress_bar, output_area]))
display(widgets.VBox([input_box,progress_bar,widgets.Output()]))